In [ ]:
!pip install asyncpraw nest_asyncio transformers
!pip install bertopic sentence-transformers torch tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

In [ ]:
import nest_asyncio
import asyncio
import asyncpraw
from datetime import datetime, timedelta
import pandas as pd
import nltk
from transformers import pipeline

nest_asyncio.apply()
sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
client_id = 'hmYAnMcW-lganmLRRzx7oQ'
client_secret = 'VeB6ehmLdkkHglJp55Y4YHAhLcdgxw'
user_agent = 'HNB_Reddit_Monitor_Script:v1.0 (by /u/CPG2005)'

reddit = asyncpraw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)

In [ ]:
async def get_mentions_with_sentiment(query_terms, after_days=90, limit=5000):
    after_timestamp = int((datetime.utcnow() - timedelta(days=after_days)).timestamp())

    subreddit = await reddit.subreddit("srilanka")
    mentions = []
    processed_submission_ids = set()

    # Define the mapping from RoBERTa's labels to common sentiment words
    sentiment_label_map = {
        'LABEL_0': 'negative',
        'LABEL_1': 'neutral',
        'LABEL_2': 'positive'
    }

    async for submission in subreddit.new(limit=limit):
        if submission.created_utc < after_timestamp:
            continue

        submission_text = f"{submission.title} {submission.selftext}".lower()
        if any(term.lower() in submission_text for term in query_terms):
            if submission.id not in processed_submission_ids:
                combined_text = submission.title + "\n" + submission.selftext
                sentiment_result = sentiment_pipeline(combined_text, truncation=True, max_length=512)[0]


                # Get the score for the predicted label
                sentiment_score = sentiment_result['score']
                # Get the RoBERTa label (e.g., 'LABEL_0', 'LABEL_1', 'LABEL_2')
                roberta_label = sentiment_result['label']
                # Map to 'negative', 'neutral', 'positive'
                mapped_sentiment_label = sentiment_label_map.get(roberta_label, 'unknown')

                mentions.append({
                    'type': 'post',
                    'id': submission.id,
                    'parent_id': None,
                    'title': submission.title,
                    'selftext': submission.selftext,
                    #'num_comments': submission.num_comments,
                    'subreddit': 'srilanka',
                    'sentiment_score': sentiment_score, # Keep the raw score
                    'sentiment_label': mapped_sentiment_label, # Use the mapped label
                    #'score': submission.score,
                    'created_utc': datetime.utcfromtimestamp(submission.created_utc),
                    'url': submission.url
                })
                processed_submission_ids.add(submission.id)

        await submission.load()
        await submission.comments.replace_more(limit=None)

        for comment in submission.comments.list():
            if comment.created_utc < after_timestamp:
                continue

            comment_text = comment.body.lower()
            if any(term.lower() in comment_text for term in query_terms):
                if submission.id not in processed_submission_ids:
                    combined_text = submission.title + "\n" + submission.selftext
                    post_sentiment_result = sentiment_pipeline(combined_text, truncation=True, max_length=512)[0]
                    post_sentiment_score = post_sentiment_result['score']
                    post_roberta_label = post_sentiment_result['label']
                    post_mapped_sentiment_label = sentiment_label_map.get(post_roberta_label, 'unknown')

                    mentions.append({
                        'type': 'post',
                        'id': submission.id,
                        'parent_id': None,
                        'title': submission.title,
                        'selftext': submission.selftext,
                        #'num_comments': submission.num_comments,
                        'subreddit': 'srilanka',
                        'sentiment_score': post_sentiment_score,
                        'sentiment_label': post_mapped_sentiment_label,
                        #'score': submission.score,
                        'created_utc': datetime.utcfromtimestamp(submission.created_utc),
                        'url': submission.url
                    })
                    processed_submission_ids.add(submission.id)

                comment_sentiment_result = sentiment_pipeline(comment.body, truncation=True, max_length=512)[0]
                comment_sentiment_score = comment_sentiment_result['score']
                comment_roberta_label = comment_sentiment_result['label']
                comment_mapped_sentiment_label = sentiment_label_map.get(comment_roberta_label, 'unknown')

                mentions.append({
                    'type': 'comment',
                    'id': comment.id,
                    'parent_id': submission.id,
                    'post_title': submission.title,
                    'selftext': comment.body,
                    #'num_comments': None,
                    'subreddit': 'srilanka',
                    'sentiment_score': comment_sentiment_score,
                    'sentiment_label': comment_mapped_sentiment_label,
                    #'score': comment.score,
                    'created_utc': datetime.utcfromtimestamp(comment.created_utc),
                    'url': f"https://reddit.com{comment.permalink}"
                })

    return mentions

In [ ]:
def results_Bank(query_terms):
  results = asyncio.run(get_mentions_with_sentiment(query_terms,after_days=90, limit=5000))

  df = pd.DataFrame(results)

  # DEBUG: Check coverage
  print("Earliest post/comment date retrieved:", df['created_utc'].min())

  if df.empty:
    print("No data in past 90 days. Trying past 1 year...")
    results = asyncio.run(get_mentions_with_sentiment(after_days=365, limit=1000))  # Increase for fallback
    df = pd.DataFrame(results)

  return df

In [ ]:
query_terms = ['HNB', 'Hatton National Bank', 'HNB Bank']
#query_terms = ['Sampath','Sampath Bank']
#query_terms = ["People's Bank","Peoples Bank"]

df = results_Bank(query_terms)
df.head()

Earliest post/comment date retrieved: 2025-07-04 08:49:51


,type,id,parent_id,title,selftext,subreddit,sentiment_score,sentiment_label,created_utc,url,post_title
0,post,1m1wzdq,None,Best savings account for an 18 y/o looking to ...,Hi everyone! I’m 18 and I’ve already had a sav...,srilanka,0.475791,neutral,2025-07-17 03:24:05,https://www.reddit.com/r/srilanka/comments/1m1...,NaN
1,comment,n3khorr,1m1wzdq,NaN,Savings accounts aren't going to give you good...,srilanka,0.591601,neutral,2025-07-17 03:36:39,https://reddit.com/r/srilanka/comments/1m1wzdq...,Best savings account for an 18 y/o looking to ...
2,post,1m1q06l,None,Anyone know why I cant make any International ...,I tried buying some games in steam and also tr...,srilanka,0.550500,negative,2025-07-16 22:03:44,https://www.reddit.com/r/srilanka/comments/1m1...,NaN
3,comment,n3jkwo1,1m1q06l,NaN,HNB doesnt do dollar payments. Sampath should ...,srilanka,0.551738,neutral,2025-07-17 00:13:46,https://reddit.com/r/srilanka/comments/1m1q06l...,Anyone know why I cant make any International ...
4,post,1lzk7qg,None,Opening PFC/NRFC account ..,"To open the best PFC/NRFC account, which bank ...",srilanka,0.701721,neutral,2025-07-14 11:45:15,https://www.reddit.com/r/srilanka/comments/1lz...,NaN


In [ ]:
import json
import csv
from collections import defaultdict

def group_mentions_from_csv(csv_file_path, json_output_path):
    grouped = defaultdict(lambda: {"post": {}, "comments": []})

    with open(csv_file_path, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)

        for row in reader:
            if row['type'] == 'post':
                post_id = row['id']
                post_content = row['title']
                post_text = row['selftext']
                grouped[post_id]["post"] = {
                    "title": post_content,
                    "selftext": post_text
                }
            elif row['type'] == 'comment':
                parent_id = row['parent_id']
                comment_text = row['selftext']
                grouped[parent_id]["comments"].append(comment_text)

    grouped_list = list(grouped.values())

    with open(json_output_path, mode='w', encoding='utf-8') as jsonfile:
        json.dump(grouped_list, jsonfile, ensure_ascii=False, indent=2)

In [ ]:
print("Df converted to csv")
df.to_csv(f"{query_terms[0]}_posts_comments.csv", index=False)
print("Group into Json")
group_mentions_from_csv(f"/content/{query_terms[0]}_posts_comments.csv", f"{query_terms[0]}_grouped_mentions.json")

Df converted to csv
Group into Json


In [ ]:
print(f"Total Mentions: {len(df)}")
print(f"Post: {len(df[df['type'] == 'post'])}")
print(f"Comments: {len(df[df['type'] == 'comment'])}")

# # Sentiment Summary
# df['sentiment_label'] = df['sentiment'].apply(lambda x: 'positive' if x > 0.05 else ('negative' if x < -0.05 else 'neutral'))
# print("\n📈 Sentiment Breakdown:")
# print(df['sentiment_label'].value_counts())

Total Mentions: 26
Post: 11
Comments: 15



**Adding Topic and Intent**



In [ ]:
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from tqdm import tqdm

# Load CSV
#df = pd.read_csv(f"/content/{query_terms[0]}_posts_comments.csv")
df = pd.read_csv("/content/HNB_Reviews - Reddit_1_HNB.csv")

# Fill NaNs in text fields
df['title'] = df['title'].fillna('')
df['selftext'] = df['selftext'].fillna('')
df['post_title'] = df['post_title'].fillna('')

# Combine relevant text fields for analysis
df['full_text'] = df.apply(lambda row: row['title'] + " " + row['selftext'] if row['type'] == 'post' else row['post_title'], axis=1)


In [ ]:
# Model for predicting topic and intent using BART
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=-1)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:

print("Running Topic Prediction ...")

# embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # or "paraphrase-MiniLM-L6-v2"
# topic_model = BERTopic(embedding_model=embedding_model, verbose=True)
# topics, _ = topic_model.fit_transform(df['full_text'].tolist())

topic_mapping = [
    "Digital Banking & E-Services", "Customer Service", "General","Account Management & Financial Products",
    "Security & Fraud Concerns","In-Branch Experience & ATM Issues","Other" ]

# Predict intent for each post/comment
topics = []
for text in tqdm(df['full_text'].tolist(), desc="Classifying topics"):
    result_topic = classifier(text, candidate_labels=topic_mapping)
    topics.append(result_topic['labels'][0])  # Top predicted intent

df['topic'] = topics


Running Topic Prediction ...


Classifying topics: 100%|██████████| 64/64 [08:22<00:00,  7.86s/it]


In [ ]:

print("Running Intent Prediction ...")

# candidate intents
candidate_intents = [
    "Positive Feedback","Query", "complaint", "discussion", "advice", "news", "experience", "recommendation", "personal story", "promotion","General Comment"
]

# Predict intent for each post/comment
intents = []
for text in tqdm(df['full_text'].tolist(), desc="Classifying intents"):
    result_intent = classifier(text, candidate_labels=candidate_intents)
    intents.append(result_intent['labels'][0])  # Top predicted intent

df['intent'] = intents


Running Intent Prediction ...


Classifying intents: 100%|██████████| 64/64 [12:39<00:00, 11.86s/it]


In [ ]:
# ----------------------------------------------------
# Save Enhanced CSV
# ----------------------------------------------------
# df.to_csv(f"/content/{query_terms[0]}_posts_comments_enriched.csv", index=False)
# print(f"✅ Saved enriched data to {query_terms[0]}_posts_comments_enriched.csv'")

df.to_csv("posts_comments_enriched.csv", index=False)
print(f"✅ Saved enriched data to posts_comments_enriched.csv'")


✅ Saved enriched data to posts_comments_enriched.csv'
